In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.interpolate import griddata
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('logs/old/analyseParameters_0npzOnly_filtered.csv')
df.shape

In [ ]:
def parallel_coordinates(df, y, filename):
    _df = df[df['filename'] == filename]
    # Create a parallel coordinates plot
    fig = px.parallel_coordinates(_df, dimensions=['seq_len', 'gen_len', 'mem_len', y],
                                  color=y, color_continuous_scale=px.colors.sequential.Viridis, title=filename)
    fig.show()
    #fig.write_image(f"../../writing/images/analyseParameter{y}{filename}.png")

def moving_plot(df, y):
    # Create 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=df['seq_len'],
        y=df['gen_len'],
        z=df['mem_len'],
        mode='markers',
        marker=dict(
            size=5,
            color=df[y],  # Color by `acc_sound`
            colorscale='Viridis',
            colorbar=dict(title=y)
        )
    )])
    fig.update_layout(scene=dict(
        xaxis_title='seq_len',
        yaxis_title='gen_len',
        zaxis_title='mem_len'),
        title='3D Scatter Plot of ' + y)
    fig.show()

def color_plot(df, y):
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Heatmap for seq_len vs. gen_len, aggregated over mem_len
    pivot_seq_gen = df.groupby(['seq_len', 'gen_len'])[y].mean().unstack()
    sns.heatmap(pivot_seq_gen, ax=axes[0], cmap="viridis")
    axes[0].set_title(y + ' by seq_len and gen_len')
    axes[0].set_xlabel('gen_len')
    axes[0].set_ylabel('seq_len')
    axes[0].invert_yaxis()  # Start y-axis from lowest to highest

    # Heatmap for seq_len vs. mem_len, aggregated over gen_len
    pivot_seq_mem = df.groupby(['seq_len', 'mem_len'])[y].mean().unstack()
    sns.heatmap(pivot_seq_mem, ax=axes[1], cmap="viridis")
    axes[1].set_title(y + ' by seq_len and mem_len')
    axes[1].set_xlabel('mem_len')
    axes[1].set_ylabel('seq_len')
    axes[1].invert_yaxis()  # Start y-axis from lowest to highest

    # Heatmap for gen_len vs. mem_len, aggregated over seq_len
    pivot_gen_mem = df.groupby(['gen_len', 'mem_len'])[y].mean().unstack()
    sns.heatmap(pivot_gen_mem, ax=axes[2], cmap="viridis")
    axes[2].set_title(y + ' by gen_len and mem_len')
    axes[2].set_xlabel('mem_len')
    axes[2].set_ylabel('gen_len')
    axes[2].invert_yaxis()  # Start y-axis from lowest to highest

    plt.tight_layout()
    plt.show()


In [ ]:
parallel_coordinates(df, 'acc_sound', '0.npz')
parallel_coordinates(df, 'acc_delta', '0.npz')
#moving_plot(_df, 'acc_sound')
#moving_plot(_df, 'acc_delta')
#color_plot(_df, 'acc_sound')
#color_plot(_df, 'acc_delta')
